In [1]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2
from PIL import Image
import os

model = load_model("groundingdino/config/GroundingDINO_SwinB_cfg.py", "weights/groundingdino_swinb_cogcoor.pth")

C:\Users\lenar\Documents\GroundingDINO\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [2]:
def get_bb(IMAGE_PATH):
    image_source, image = load_image(IMAGE_PATH)
    
    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )
    
    annotated_frame, bbs = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
    return bbs, image_source

In [8]:
import cv2
import mediapipe as mp
import numpy as np
import math

mp_hands = mp.solutions.hands

def calculate_angle(p1, p2):
    """Вычисляет угол между линией, соединяющей две точки, и вертикальной осью (ось Y)."""
    delta_x = p2[0] - p1[0]
    delta_y = p2[1] - p1[1]
    angle_rad = math.atan2(delta_y, delta_x)
    return 90 + math.degrees(angle_rad)

def rotate_image(image, angle):
    """Поворачивает изображение на заданный угол без обрезки."""
    height, width = image.shape[:2]

    # Вычисляем диагональ изображения
    diagonal = int(math.sqrt(width ** 2 + height ** 2))

    # Создаем увеличенное изображение с размерами диагонали, чтобы избежать обрезки
    enlarged_image = cv2.copyMakeBorder(image, (diagonal - height) // 2,
                                        (diagonal - height) // 2,
                                        (diagonal - width) // 2,
                                        (diagonal - width) // 2,
                                        cv2.BORDER_CONSTANT, value=[0, 0, 0])

    # Поворачиваем изображение
    rotation_matrix = cv2.getRotationMatrix2D((diagonal // 2, diagonal // 2), angle, 1)
    rotated_image = cv2.warpAffine(enlarged_image, rotation_matrix, (diagonal, diagonal))

    # Обрезка изображения до размера диагонали
    y_offset = (rotated_image.shape[0] - diagonal) // 2
    x_offset = (rotated_image.shape[1] - diagonal) // 2
    cropped_image = rotated_image[y_offset:y_offset + diagonal, x_offset:x_offset + diagonal]

    return cropped_image

def correct_hand_orientation(image):
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if not results.multi_hand_landmarks:
            print("Рука не обнаружена")
            return image

        for hand_landmarks in results.multi_hand_landmarks:
            # Получаем координаты точек 0 и 12
            x0, y0 = hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y
            x12, y12 = hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y

            # Переводим в пиксели
            height, width, _ = image.shape
            p0 = (int(x0 * width), int(y0 * height))
            p12 = (int(x12 * width), int(y12 * height))

            # Вычисляем угол между линией (0, 12) и вертикальной осью
            angle = calculate_angle(p0, p12)

            # Поворачиваем изображение на полученный угол
            corrected_image = rotate_image(image, angle)

            return corrected_image

    return image

def correct_hand_orientation(image):
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if not results.multi_hand_landmarks:
            print("Рука не обнаружена")
            return image

        for hand_landmarks in results.multi_hand_landmarks:
            # Получаем координаты точек 0 и 12
            x0, y0 = hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y
            x12, y12 = hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y

            # Переводим в пиксели
            height, width, _ = image.shape
            p0 = (int(x0 * width), int(y0 * height))
            p12 = (int(x12 * width), int(y12 * height))

            # Вычисляем угол между линией (0, 12) и вертикальной осью
            angle = calculate_angle(p0, p12)

            # Поворачиваем изображение на полученный угол
            corrected_image = rotate_image(image, angle)

            return corrected_image

    return image

def get_p0(image):
    with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5) as hands:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if not results.multi_hand_landmarks:
            print("Рука не обнаружена")
            return None  # Возвращает None, если рука не обнаружена

        for hand_landmarks in results.multi_hand_landmarks:
            # Получаем координаты точки 0
            x0, y0 = hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y

            # Переводим в пиксели
            height, width, _ = image.shape
            p0 = (int(x0 * width), int(y0 * height))

            return p0  # Возвращаем только координаты p0

In [2]:
dir_path = 'path-to-images-palm'

In [7]:
TEXT_PROMPT = "palm"
BOX_TRESHOLD = 0.3
TEXT_TRESHOLD = 0.3

for img_name in os.listdir(dir_path):
    IMAGE_PATH = dir_path + img_name
    bbs, image_source = get_bb(IMAGE_PATH)
    for i, bb in enumerate(bbs):
        bb = [int(b) for b in bb]
        image = cv2.cvtColor(image_source[bb[1]:bb[3],bb[0]:bb[2]], cv2.COLOR_BGR2RGB)
        corrected_image = correct_hand_orientation(image)
        cv2.imwrite('path-to-cropped-palm/' + str(i) + img_name, corrected_image)

TypeError: 'NoneType' object is not iterable

In [8]:
from PIL import Image, ImageOps

def make_square(image_path, output_path):
    with Image.open(image_path) as im:
        width, height = im.size

        # Определяем, сколько пикселей нужно добавить
        if width > height:
            delta = width - height
            padding = (0, delta // 2, 0, delta - delta // 2)
        else:
            delta = height - width
            padding = (delta // 2, 0, delta - delta // 2, 0)

        # Добавляем паддинг и создаем квадратное изображение
        new_im = ImageOps.expand(im, padding)
        new_im.save(output_path)


In [9]:
folder = 'path-to-cropped-palm/'

In [10]:
for img in os.listdir(folder):
    make_square(folder + img, 'palm/' + img)